Using https://ensemble-pytorch.readthedocs.io/en/latest/parameters.html#voting

In [1]:
import sys

sys.path.insert(1, "../")

from nets.SimpleNet.lib.simple_model import CNN
from nets.SimpleNet.config.torch_config import get_transform
from nets.SimpleNet.utils.video_dataset import VideoFrameDataset
from nets.SimpleNet.config.dataset import get_dataset_path
from nets.SimpleNet.utils.loader import split_dataset


In [2]:
NUM_SEGMENTS = 10
FRAMES_PER_SEGMENT = 5
BATCH_SIZE = 16
IMAGE_SIZE = 64
IMAGE_RANDOM_CROP_RESIZE = 0.8
NUM_EPOCHS = 5
LEARNING_RATE = 1e-3
DEBUG = False



In [3]:
data_path, model_path = get_dataset_path(dataset="WLASL/videos", model_name="WLASL")

In [4]:
multiple_transform = get_transform(IMAGE_SIZE, IMAGE_RANDOM_CROP_RESIZE)

In [5]:
dataset = VideoFrameDataset(
    root_path=data_path,
    transform=multiple_transform,
    num_segments=NUM_SEGMENTS,
    frames_per_segment=FRAMES_PER_SEGMENT,
)

classes = dataset.classes
print(classes)


['before', 'book', 'candy', 'chair', 'clothes']


In [6]:
train_loader, test_loader, validation_loader = split_dataset(
    dataset, train_split=0.70, validation_split=0.1, batch_size=BATCH_SIZE
)


In [7]:
from torchensemble import VotingClassifier

ensemble = VotingClassifier(
    estimator=CNN,
    estimator_args={
        "num_classes": len(classes),
        "batch_size": BATCH_SIZE,
        "num_frames": FRAMES_PER_SEGMENT * NUM_SEGMENTS,
        "image_size": int(IMAGE_SIZE * IMAGE_RANDOM_CROP_RESIZE),
        "debug": DEBUG,
    },
    n_estimators=1,
    cuda=True,
)


In [25]:
from torch import nn, optim
from torchensemble.utils.set_module import set_scheduler

In [26]:
ensemble.set_criterion(nn.CrossEntropyLoss())

In [28]:
# optimizer = optim.Adam(ensemble.parameters(), lr=LEARNING_RATE)
ensemble.set_optimizer("Adam", lr=LEARNING_RATE)

In [37]:
ensemble.set_scheduler(
    "ReduceLROnPlateau",
    optimizer=optim.__dict__[ensemble.optimizer_name](
        ensemble.parameters(), lr=ensemble.optimizer_args["lr"]
    ),
    mode="min",
    min_lr=1e-6,
    factor=0.7,
    patience=5,
)


ValueError: optimizer got an empty parameter list

In [34]:
ensemble.fit(
    train_loader=train_loader,
    test_loader=validation_loader,
    epochs=NUM_EPOCHS,
)


AttributeError: 'ReduceLROnPlateau' object has no attribute 'get_last_lr'

In [ ]:
accuracy = ensemble.evaluate(test_loader)
print(f"Accuracy: {accuracy}")


Accuracy: 44.44444444444444
